# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
import gluonts
gluonts.__version__

'0.4.0'

In [2]:
# !pip install gluonts

In [3]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [7]:
def deepar(data="m4_quarterly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        context_length=dataset.metadata.prediction_length*2,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [8]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=100, batches=50)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.09it/s, avg_epoch_loss=8.21]
INFO:root:Epoch[0] Elapsed time 9.847 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.207497
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.55it/s, avg_epoch_loss=7.6]
INFO:root:Epoch[1] Elapsed time 9.028 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.597656
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.63it/s, avg_epoch_loss=7.51]
INFO:root:Epoch[2] Elapsed time 6.559 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.510768
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.28it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[3] Elapsed time 6.878 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.272761
INFO:

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  9.26it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[36] Elapsed time 5.408 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.860831
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  9.06it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[37] Elapsed time 5.528 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.991605
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  9.27it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[38] Elapsed time 5.402 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.781770
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  9.40it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[39] Elapsed time 5.327 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.862946
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:05<00:00,  8.82it/s, avg_epoch_loss=6.99]
INFO:ro

INFO:root:Epoch[72] Elapsed time 5.501 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.769806
INFO:root:Epoch[73] Learning rate is 0.000125
100%|██████████| 50/50 [00:06<00:00,  7.27it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[73] Elapsed time 6.886 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.795686
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:07<00:00,  6.80it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[74] Elapsed time 7.364 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.979128
INFO:root:Loading parameters from best epoch (64)
INFO:root:Epoch[75] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:08<00:00,  5.76it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[75] Elapsed time 8.700 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.605432
INFO:root:Epoch[76] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:05<00:00,  9.02it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[76] Elapsed time 5.551 seconds
INFO:root:Epoch[

{'MASE': 1.34675324,
 'MSIS': 17.6553076,
 'epochs': 100,
 'sMAPE': 0.11075865,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.10363214,
 'wQuantileLoss[0.9]': 0.06326584}
Wall time: 38min 32s


In [9]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=200, batches=50)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.60it/s, avg_epoch_loss=8.21]
INFO:root:Epoch[0] Elapsed time 13.906 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.207497
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.45it/s, avg_epoch_loss=7.6]
INFO:root:Epoch[1] Elapsed time 11.245 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.597656
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:17<00:00,  2.81it/s, avg_epoch_loss=7.51]
INFO:root:Epoch[2] Elapsed time 17.816 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.510768
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:16<00:00,  2.94it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[3] Elapsed time 17.017 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.272761
I

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 50/50 [00:31<00:00,  1.60it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[36] Elapsed time 31.368 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.860831
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:30<00:00,  1.64it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[37] Elapsed time 30.479 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.991605
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:27<00:00,  1.83it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[38] Elapsed time 27.463 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.781770
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:13<00:00,  3.70it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[39] Elapsed time 13.550 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.862946
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:20<00:00,  2.49it/s, avg_epoch_loss=6.99]
INF

100%|██████████| 50/50 [00:12<00:00,  3.90it/s, avg_epoch_loss=6.77]
INFO:root:Epoch[72] Elapsed time 12.841 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.769806
INFO:root:Epoch[73] Learning rate is 0.000125
100%|██████████| 50/50 [00:07<00:00,  6.54it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[73] Elapsed time 7.658 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.795686
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:07<00:00,  6.52it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[74] Elapsed time 7.675 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.979128
INFO:root:Loading parameters from best epoch (64)
INFO:root:Epoch[75] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:07<00:00,  6.33it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[75] Elapsed time 7.902 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.605432
INFO:root:Epoch[76] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:08<00:00,  6.05it/s, avg_epoch_loss

INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=6.572338
INFO:root:Epoch[109] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.74it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[109] Elapsed time 7.432 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=6.445246
INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.42it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[110] Elapsed time 9.242 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.974647
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.46it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[111] Elapsed time 7.757 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.807831
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.27it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[112] Elapsed time 7.982 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.919374
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██

INFO:root:Epoch[145] Elapsed time 8.241 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=6.801136
INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.54it/s, avg_epoch_loss=7.3]
INFO:root:Epoch[146] Elapsed time 7.647 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=7.302226
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.57it/s, avg_epoch_loss=6.64]
INFO:root:Epoch[147] Elapsed time 7.615 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.640840
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  7.14it/s, avg_epoch_loss=6.68]
INFO:root:Epoch[148] Elapsed time 7.013 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=6.679082
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.73it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[149] Elapsed time 7.433 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=6.917677
INFO

100%|██████████| 50/50 [00:07<00:00,  7.03it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[182] Elapsed time 7.114 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=6.988324
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 50/50 [00:08<00:00,  5.82it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[183] Elapsed time 8.594 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.439885
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 50/50 [00:06<00:00,  7.97it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[184] Elapsed time 6.291 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=6.400814
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.31it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[185] Elapsed time 7.937 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=6.713808
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  8.79it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[186] Elapsed time 5.700 seco

{'MASE': 1.33229542,
 'MSIS': 15.53920039,
 'epochs': 200,
 'sMAPE': 0.10991944,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.10204718,
 'wQuantileLoss[0.9]': 0.06041322}
Wall time: 1h 5min 30s


In [10]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=300, batches=50)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.48it/s, avg_epoch_loss=8.21]
INFO:root:Epoch[0] Elapsed time 11.196 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.207497
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:08<00:00,  5.65it/s, avg_epoch_loss=7.6]
INFO:root:Epoch[1] Elapsed time 8.860 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.597656
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.98it/s, avg_epoch_loss=7.51]
INFO:root:Epoch[2] Elapsed time 10.114 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.510768
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.22it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[3] Elapsed time 11.878 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.272761
IN

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 50/50 [00:06<00:00,  7.61it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[36] Elapsed time 6.575 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.860831
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:06<00:00,  7.16it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[37] Elapsed time 6.999 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.991605
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.84it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[38] Elapsed time 10.335 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.781770
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.79it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[39] Elapsed time 10.451 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.862946
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:21<00:00,  2.28it/s, avg_epoch_loss=6.99]
INFO:

INFO:root:Epoch[72] Elapsed time 5.951 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.769806
INFO:root:Epoch[73] Learning rate is 0.000125
100%|██████████| 50/50 [00:10<00:00,  4.81it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[73] Elapsed time 10.394 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.795686
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:07<00:00,  6.92it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[74] Elapsed time 7.233 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.979128
INFO:root:Loading parameters from best epoch (64)
INFO:root:Epoch[75] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:05<00:00,  9.06it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[75] Elapsed time 5.528 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.605432
INFO:root:Epoch[76] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:05<00:00,  8.82it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[76] Elapsed time 5.680 seconds
INFO:root:Epoch

INFO:root:Epoch[109] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.05it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[109] Elapsed time 3.837 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=6.445246
INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.07it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[110] Elapsed time 3.831 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.974647
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.59it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[111] Elapsed time 3.687 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.807831
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.31it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[112] Elapsed time 3.763 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=6.919374
INFO:root:Epoch[113] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.41it/s, avg_epoch_loss=6.72]

INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=6.801136
INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.28it/s, avg_epoch_loss=7.3]
INFO:root:Epoch[146] Elapsed time 3.771 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=7.302226
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.57it/s, avg_epoch_loss=6.64]
INFO:root:Epoch[147] Elapsed time 3.693 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.640840
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.42it/s, avg_epoch_loss=6.68]
INFO:root:Epoch[148] Elapsed time 4.031 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=6.679082
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.50it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[149] Elapsed time 4.354 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=6.917677
INFO:root:Epoch[150] Learning rate is 5e-05
100%|███

INFO:root:Epoch[182] Elapsed time 3.608 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=6.988324
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.92it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[183] Elapsed time 4.585 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.439885
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.42it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[184] Elapsed time 4.803 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=6.400814
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.33it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[185] Elapsed time 4.064 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=6.713808
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 50/50 [00:06<00:00,  8.13it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[186] Elapsed time 6.157 seconds
INFO:root:Epoch[186] Evaluation metric 'epoch_loss'=6.812138
INFO

100%|██████████| 50/50 [00:03<00:00, 13.25it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[219] Elapsed time 3.780 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=6.623721
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.93it/s, avg_epoch_loss=6.77]
INFO:root:Epoch[220] Elapsed time 3.596 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=6.773580
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.27it/s, avg_epoch_loss=7.06]
INFO:root:Epoch[221] Elapsed time 3.773 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=7.055017
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.50it/s, avg_epoch_loss=6.55]
INFO:root:Epoch[222] Elapsed time 3.711 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=6.545161
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.48it/s, avg_epoch_loss=6.65]
INFO:root:Epoch[223] Elapsed time 3.716 sec

INFO:root:Epoch[256] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.25it/s, avg_epoch_loss=6.59]
INFO:root:Epoch[256] Elapsed time 3.779 seconds
INFO:root:Epoch[256] Evaluation metric 'epoch_loss'=6.588757
INFO:root:Epoch[257] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.62it/s, avg_epoch_loss=6.84]
INFO:root:Epoch[257] Elapsed time 3.678 seconds
INFO:root:Epoch[257] Evaluation metric 'epoch_loss'=6.842503
INFO:root:Epoch[258] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.87it/s, avg_epoch_loss=6.68]
INFO:root:Epoch[258] Elapsed time 3.611 seconds
INFO:root:Epoch[258] Evaluation metric 'epoch_loss'=6.676251
INFO:root:Epoch[259] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.55it/s, avg_epoch_loss=6.33]
INFO:root:Epoch[259] Elapsed time 3.695 seconds
INFO:root:Epoch[259] Evaluation metric 'epoch_loss'=6.331484
INFO:root:Epoch[260] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 12.55it/s, avg_epoch_loss=6.5]


INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=6.643747
INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.56it/s, avg_epoch_loss=6.79]
INFO:root:Epoch[293] Elapsed time 3.695 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.790904
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.75it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[294] Elapsed time 3.641 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.392891
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.53it/s, avg_epoch_loss=6.67]
INFO:root:Epoch[295] Elapsed time 3.704 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=6.674220
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 13.33it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[296] Elapsed time 3.756 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=6.831643
INFO:root:Epoch[297] Learning rate is 5e-05
100%|██

{'MASE': 1.33249243,
 'MSIS': 15.5425546,
 'epochs': 300,
 'sMAPE': 0.10998624,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.10208643,
 'wQuantileLoss[0.9]': 0.06047772}
Wall time: 43min
Parser   : 977 ms


In [11]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=400, batches=50)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.87it/s, avg_epoch_loss=8.21]
INFO:root:Epoch[0] Elapsed time 4.220 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.207497
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.20it/s, avg_epoch_loss=7.6]
INFO:root:Epoch[1] Elapsed time 4.105 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.597656
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 13.02it/s, avg_epoch_loss=7.51]
INFO:root:Epoch[2] Elapsed time 3.848 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.510768
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 12.74it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[3] Elapsed time 3.928 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.272761
INFO:

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 11.70it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[36] Elapsed time 4.283 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.860831
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 10.82it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[37] Elapsed time 4.630 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.991605
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 11.44it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[38] Elapsed time 4.378 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.781770
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 10.80it/s, avg_epoch_loss=6.86]
INFO:root:Epoch[39] Elapsed time 4.647 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.862946
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 11.63it/s, avg_epoch_loss=6.99]
INFO:ro

KeyboardInterrupt: 

In [12]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=500, batches=50)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/50 [00:02<?, ?it/s, avg_epoch_loss=8.79]


KeyboardInterrupt: 

In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results

### DeepAR - use_feat_static==False

In [7]:
def deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
#     cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)
